# Preprocessing UFOs

## Importing and checking data types

In [1]:
import pandas as pd

ufo_path = '~/Desktop/STA 141B/141B-Final-Project/ufo-sightings/scrubbed.csv'

# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
ufos = pd.read_csv(ufo_path, header=0,
                   names=['date_seen', 'city', 'state_abbr', 'country', 'shape', 'duration_sec',
                          'duration_min', 'comments', 'date_posted', 'latitude', 'longitude'],
                   parse_dates=['date_posted'], infer_datetime_format=True)

# check data types
ufos.info()

//anaconda/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
date_seen       80332 non-null object
city            80332 non-null object
state_abbr      74535 non-null object
country         70662 non-null object
shape           78400 non-null object
duration_sec    80332 non-null object
duration_min    80332 non-null object
comments        80317 non-null object
date_posted     80332 non-null datetime64[ns]
latitude        80332 non-null object
longitude       80332 non-null float64
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 6.7+ MB


## Change columns data type

In [2]:
# change duration in seconds and latitude to numeric
ufos[['duration_sec', 'latitude']] = ufos[['duration_sec', 'latitude']].apply(pd.to_numeric)

ValueError: ('Unable to parse string "2`" at position 27822', u'occurred at index duration_sec')

In [3]:
# find the errors
print ufos[ufos['duration_sec'].str.contains('`', na=False)]

# from the result we see that we need to set values to 3 rows of this instance
ufos = ufos.set_value(27822, 'duration_sec', '2')
ufos = ufos.set_value(35692, 'duration_sec', '8')
ufos = ufos.set_value(58591, 'duration_sec', '0.5')

             date_seen               city state_abbr country   shape  \
27822   2/2/2000 19:33              bouse         az      us     NaN   
35692  4/10/2005 22:52         santa cruz         ca      us     NaN   
58591  7/21/2006 13:00  ibague (colombia)        NaN     NaN  circle   

      duration_sec        duration_min  \
27822           2`  each a few seconds   
35692           8`       eight seconds   
58591         0.5`         1/2 segundo   

                                                comments date_posted  \
27822  Driving through Plomosa Pass towards Bouse Loo...  2000-02-16   
35692  2 red lights moving together and apart with a ...  2005-04-16   
58591  Viajaba a 27.000 pies en un avion comercial ve...  2006-10-30   

         latitude   longitude  
27822  33.9325000 -114.005000  
35692  36.9741667 -122.029722  
58591    4.440663  -75.244141  


In [4]:
# try again
ufos[['duration_sec', 'latitude']] = ufos[['duration_sec', 'latitude']].apply(pd.to_numeric)

ValueError: ('Unable to parse string "33q.200088" at position 43782', u'occurred at index latitude')

In [5]:
print ufos[ufos['latitude'].str.contains('[a-zA-Z]', na=False)]

# we again see that lat has a letter in one of the positions we'll check to
# see how many there are
ufos = ufos.set_value(43782, 'latitude', '33.200088')

             date_seen                          city state_abbr country  \
43782  5/22/1974 05:30  mescalero indian reservation         nm     NaN   

           shape duration_sec duration_min  \
43782  rectangle          180    two hours   

                                                comments date_posted  \
43782  Huge rectangular object emmitting intense whit...  2012-04-18   

         latitude   longitude  
43782  33q.200088 -105.624152  


### Update duration_min from duration_sec

In [6]:
# change duration in seconds and latitude to numeric
ufos[['duration_sec', 'latitude']] = ufos[['duration_sec', 'latitude']].apply(pd.to_numeric)

# change seconds to minutes
ufos['duration_min'] =  ufos['duration_sec']/60

# drop the seconds column, we can get minutes from seconds
del ufos['duration_sec']

## Reformat names of records & set DateTime index

In [7]:
# upper case words as necessary
ufos['city'] = ufos['city'].str.title()
ufos['state_abbr'] = ufos['state_abbr'].str.upper()
ufos['country'] = ufos['country'].str.upper()

# change date_seen col to date_time type
ufos['date_seen'] = pd.to_datetime(ufos['date_seen'], errors='coerce')

# set row indices as date_seen
ufos = ufos.set_index('date_seen')

# check how many US records there are
print ufos[ufos['country'] == 'US'].shape

(65114, 9)


## Explore for anomalies

In [8]:
# want to subet by country, but check if there is any NA we can change
na_country = ufos[ufos['state_abbr'].notnull() & ufos['country'].isnull()]
print na_country.shape
na_country.head()

(6414, 9)


,city,state_abbr,country,shape,duration_min,comments,date_posted,latitude,longitude
date_seen,,,,,,,,,
1949-10-10 21:00:00,Lackland Afb,TX,NaN,light,120.0,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
1979-10-10 22:00:00,Saddle Lake (Canada),AB,NaN,triangle,4.5,Lights far above&#44 that glance; then flee f...,2005-01-19,53.970571,-111.689885
1986-10-10 20:00:00,Holmes/Pawling,NY,NaN,chevron,3.0,Football Field Sized Chevron with bright white...,2007-10-08,41.523427,-73.646795
1989-10-10 21:00:00,Kranklin,KY,NaN,triangle,3.0,Triangle seen in franklin Ky - october 1989,2005-05-11,36.722263,-86.577218
1994-10-10 15:00:00,Mercedies,TX,NaN,cigar,60.0,ufo chased by fighter jet over Rio Grande Vall...,2011-12-12,26.149798,-97.913611


From looking we can see many records in the DF that have a state (most likely in the US) but no country. We can match them to state_table.csv which was downloaded from https://statetable.com/ with the following steps

- USA
- Just include states/provinces
- No, do not include DC in the list
- No, do not include military addresses 
- Only include current states and territories 
- Do not include the US Minor Outlying Islands 
- A CSV file I can open in my office suite

### Combine state data

In [9]:
state_table = pd.read_csv('state_table.csv')
state_table.head()

,id,name,abbreviation,country,type,sort,status,occupied,notes,fips_state,assoc_press,standard_federal_region,census_region,census_region_name,census_division,census_division_name,circuit_court
0,1,Alabama,AL,USA,state,10,current,occupied,NaN,1,Ala.,IV,3,South,6,East South Central,11
1,2,Alaska,AK,USA,state,10,current,occupied,NaN,2,Alaska,X,4,West,9,Pacific,9
2,3,Arizona,AZ,USA,state,10,current,occupied,NaN,4,Ariz.,IX,4,West,8,Mountain,9
3,4,Arkansas,AR,USA,state,10,current,occupied,NaN,5,Ark.,VI,3,South,7,West South Central,8
4,5,California,CA,USA,state,10,current,occupied,NaN,6,Calif.,IX,4,West,9,Pacific,9


In [10]:
cond1 = ufos['state_abbr'].notnull() & ufos['country'].isnull()
cond2 = ufos['state_abbr'].isin(state_table['abbreviation'])     # filtering join
ufos.loc[cond1 & cond2, 'country'] = 'US'

# check how many US records there are now
print ufos[ufos['country'] == 'US'].shape

(70828, 9)


In [12]:
# check now how many records are in US but are NOT in the abbrevations list in state_table
cond1 = ufos['country'] == 'US'
cond2 = ~ufos['state_abbr'].isin(state_table['abbreviation'])
print ufos.loc[cond1 & cond2].shape
ufos.loc[cond1 & cond2]

(32, 9)


,city,state_abbr,country,shape,duration_min,comments,date_posted,latitude,longitude
date_seen,,,,,,,,,
1998-10-11 14:05:00,Vega Baja (Puerto Rico),PR,US,cigar,2.000000,I went out and saw this cigar shaped object&#4...,1998-11-01,18.446389,-66.388056
1999-01-10 01:00:00,San Lorenzo (Puerto Rico),PR,US,formation,1.000000,I was with my parent inside our car when sudde...,1999-11-02,18.191389,-65.961389
2002-11-01 23:30:00,Arroyo (Puerto Rico),PR,US,triangle,0.500000,we saw what it looked to be a triangle shap ob...,2002-11-04,17.967778,-66.061667
1989-11-15 23:00:00,Moca (Puerto Rico),PR,US,unknown,1.000000,Late&#44 cloudy night in Moca&#44 Puerto Rico....,1999-02-16,18.396667,-67.113611
2011-12-11 20:08:00,Washington,DC,US,cylinder,0.066667,Luminous object thought to be a star?,2012-01-12,38.895000,-77.036667
1997-12-31 21:05:00,Puerto Nuevo (Puerto Rico),PR,US,cross,3.500000,While visiting relatives in Puerto Rico in Dec...,1998-03-07,18.413611,-66.082222
2012-01-29 19:20:00,Washington,DC,US,other,0.166667,A group of darting/revolving round white light...,2012-02-03,38.895000,-77.036667
1974-01-31 15:00:00,Ponce (Puerto Rico),PR,US,sphere,15.000000,The spaceship was shaped like two spheres atta...,1999-02-16,18.013056,-66.614444
1998-01-31 18:00:00,Arecibo (Just S Of&#44 Rural Central Pr),PR,US,disk,15.000000,One-two hours after driving away from Arecibo ...,2002-04-25,18.474444,-66.716111


We can see these are either Purto Rico or Washington, but there's only 32 so we wont include these for the analysis

In [13]:
# subset the data to be only the US
ufos = ufos.loc[ufos['country'] == 'US']

# get records from 1953 and beyond (Natural Disasters starts from 1953 onwards)
ufos = ufos.loc['1953':]

# Here we can merge some specific columns over from the state_table to the ufos DF
stble_cols = ['abbreviation', 'name', 'census_region_name', 'census_division_name']
ufos = ufos.reset_index().merge(state_table[stble_cols], how='inner', left_on='state_abbr', right_on='abbreviation').set_index('date_seen')

In [14]:
print ufos.shape
ufos.head()

(70066, 13)


,city,state_abbr,country,shape,duration_min,comments,date_posted,latitude,longitude,abbreviation,name,census_region_name,census_division_name
date_seen,,,,,,,,,,,,,
1956-10-10 21:00:00,Edna,TX,US,circle,0.333333,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833,TX,Texas,South,West South Central
1977-10-10 12:00:00,San Antonio,TX,US,other,0.500000,i was about six or seven and my family and me ...,2005-02-24,29.423889,-98.493333,TX,Texas,South,West South Central
1980-10-10 19:00:00,Houston,TX,US,sphere,3.000000,Sphere&#44 No lights&#44 moving through neigh...,2005-04-16,29.763056,-95.363056,TX,Texas,South,West South Central
1980-10-10 22:00:00,Dallas,TX,US,unknown,5.000000,Strange shape shifting craft of pure light ene...,2002-10-28,32.783333,-96.800000,TX,Texas,South,West South Central
1984-10-10 05:00:00,Houston,TX,US,circle,1.000000,2 experience with unkown,2012-04-18,29.763056,-95.363056,TX,Texas,South,West South Central


## Output final DF

In [18]:
# delete the repeated column (abbreviation) and country (we know its in the US)
del ufos['abbreviation']
del ufos['country']

# change names of some columns
ufos = ufos.rename(columns={'name':'state', 'census_region_name':'region_name', 'census_division_name':'division_name'})

# reorder DF and sort index
reorder_cols = ['date_posted','shape','duration_min','comments', 'city', 'state', 
                'state_abbr','division_name','region_name', 'latitude', 'longitude']

ufos = ufos[reorder_cols]
ufos = ufos.sort_index()

# output the DF
ufos.to_csv('ufos.csv')

### Resources

- https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-column
- https://stackoverflow.com/questions/20375561/joining-pandas-dataframes-by-column-names
- https://stackoverflow.com/questions/11976503/how-to-keep-index-when-using-pandas-merge
- https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
- https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns
- https://stackoverflow.com/questions/20868394/changing-a-specific-column-name-in-pandas-dataframe